In [1]:
import os
import re
import csv
import codecs

In [2]:
def rename_data_file(**kwargs):
    """
    Funcao que recebe o nome do arquivo original e o renomeia, 
    com um complemento do assunto e uma nova extensao.
    
    Args:
        data_file(str): Nome do arquivo original.
            Ex.: '20110101_AC.txt'
        name_to_add(str): Nome a ser adicionado no nome do arquivo.
            Ex.: 'CNPJ'
        file_ext(str): Tipo de extensao a ser adicionado ao nome do arquivo.
            Ex.: '.csv'
    
    Returns:
        string: Nome do arquivo modificado.
            Ex.: '20110101_CNPJ_AC.csv'
    """
    # Expressao Regular para:
    # Identificar a data de referencia no nome do arquivo
    date_reference_re = re.compile(r'^[0-9]+')
    # Identificar o assunto no nome do arquivo
    file_subject_re = re.compile(r'_([\w]+)\.')
    
    # Identifica a data de referencia do arquivo
    date_reference = re.search(date_reference_re, kwargs['data_file']).group()
    # Identifica o assunto do arquivo
    subject = re.search(file_subject_re, kwargs['data_file']).group(1)
    
    # Define o novo nome do arquivo
    renamed_file = date_reference + '_' + kwargs['name_to_add'] + '_' + subject + kwargs['file_ext']
    
    return renamed_file

In [3]:
def cnpj_pos_to_col(**kwargs):
    """
    Transforma o arquivo posicional de Quadro Societario da Receita Federal em um arquivo de CNPJ delimitado.
    
    Args:
        in_data_path(str): Caminho completo do arquivo de entrada.
        out_data_dir(str): Caminho da pasta para armazenamento do arquivo de saida.
        data_file(str): Nome do arquivo de entrada.
        delimiter(str): Delimitador de campos utilizado no arquivo de saida
        file_ext(str): Extensao do arquivo de saida
        encoding(str): Codificacao do arquivo de saida
    """
    
    # Define o header para o arquivo de saida
    kwargs['header'] = 'tp_campo\tnr_cnpj\tnm_razao_social\n'
    # Define o nome a ser adicionado no arquivo de saida
    kwargs['name_to_add'] = 'CNPJ'
    
    # Nome do arquivo de saida
    kwargs['out_data_file'] = rename_data_file(**kwargs)
    # Caminho completo do arquivo de saida
    kwargs['out_data_path'] = os.path.join(kwargs['out_data_dir'], kwargs['out_data_file'])
    
    # Le o arquivo de entrada
    with codecs.open(kwargs['in_data_path'], 'r', encoding = kwargs['encoding']) as reader:
        
        # Prepara o arquivo de saida
        with codecs.open(kwargs['out_data_path'], 'w', encoding = kwargs['encoding']) as writer:
            
            # Escreve o cabecalho no inicio do arquivo de saida
            writer.write(kwargs['header'])
            
            # Para cada linha no arquivo de entrada
            for line in reader:
                
                # Se for uma linha do tipo '01 - Informacao da empresa'
                if line[:2] == "01":
                    
                    # Escreve a linha no arquivo de saida
                    writer.write(
                                                line[:2]   # tp_campo
                        + kwargs['delimiter'] + line[2:16] # nr_cnpj
                        + kwargs['delimiter'] + line[16:]  # nm_razao_social
                    )
                    
    print "Arquivo " + kwargs['data_file'] + " organizado por CNPJ."

In [4]:
def socio_pos_to_col(**kwargs):
    """
    Transforma o arquivo posicional de Quadro Societario da Receita Federal
    em um arquivo de Quadro Societario delimitado.
    
    Args:
        in_data_path(str): Caminho completo do arquivo de entrada.
        out_data_dir(str): Caminho da pasta para armazenamento do arquivo de saida.
        data_file(str): Nome do arquivo de entrada.
        delimiter(str): Delimitador de campos utilizado no arquivo de saida
        file_ext(str): Extensao do arquivo de saida
        encoding(str): Codificacao do arquivo de saida
    """
    # Define o header para o arquivo de saida separado por tabulacao
    kwargs['header'] = "tp_campo\tnr_cnpj\tin_cpf_cnpj_socio\tnr_cpf_cnpj_socio\tcd_qualificacao_socio\tnm_socio\n"
    # Define o nome a ser adicionado no arquivo de saida
    kwargs['name_to_add'] = 'Quadro'
    
    # Nome do arquivo de saida
    kwargs['out_data_file'] = rename_data_file(**kwargs)
    # Caminho completo do arquivo de saida
    kwargs['out_data_path'] = os.path.join(kwargs['out_data_dir'], kwargs['out_data_file'])
    
    # Le o arquivo de entrada
    with codecs.open(kwargs['in_data_path'], 'r', encoding = kwargs['encoding']) as reader:
        
        # Prepara o arquivo de saida
        with codecs.open(kwargs['out_data_path'], 'w', encoding = kwargs['encoding']) as writer:
            
            # Escreve o cabecalho no inicio do arquivo de saida
            writer.write(kwargs['header'])
            
            # Para cada linha no arquivo de entrada
            for line in reader:
                
                # Se for uma linha do tipo 02-Informacao do socio
                if line[:2] == "02":
                    
                    # Escreve a linha no arquivo de saida
                    writer.write(
                                                line[:2]    # tp_campo
                        + kwargs['delimiter'] + line[2:16]  # nr_cnpj
                        + kwargs['delimiter'] + line[16:17] # in_cpf_cnpj
                        + kwargs['delimiter'] + line[17:31] # nr_cpf_cnpj_socio
                        + kwargs['delimiter'] + line[31:33] # cd_qualificacao_socio
                        + kwargs['delimiter'] + line[33:]   # nm_socio
                    )
                    
    print "Arquivo " + kwargs['data_file'] + " organizado por Quadro Societario."

In [5]:
def organize_files():
    """
    Funcao que organiza o arquivo de Quadro Societario da Receita Federal 
    em um arquivo de CNPJs e outro de Quadro Societario. 
    """
    
    # Prepara o dicionario de variaveis (kwargs = keyworded arguments)
    kwargs = {}
    
    # Diretório onde são armazenados os arquivos originais e organizados
    kwargs['in_data_dir'] = '..\\..\\data\\02-cleaned'
    kwargs['out_data_dir'] = '..\\..\\data\\03-organized'
    
    # Parametros de organizacao dos arquivos delimitados
    kwargs['delimiter'] = '\t'
    kwargs['file_ext'] = '.csv'
    kwargs['encoding'] = 'utf-8'

    # Lista dos arquivos originais 
    files = os.listdir(kwargs['in_data_dir'])

    # Para cada arquivo na lista de arquivos originais
    for data_file in files:

        # Define o nome do arquivo no dicionario
        kwargs['data_file'] = data_file
        
        # Define o caminho completo de acesso ao arquivo original
        kwargs['in_data_path'] = os.path.join(kwargs['in_data_dir'], kwargs['data_file'])

        # Organiza os arquivos por CNPJ
        cnpj_pos_to_col(**kwargs)
        
        # Organiza os arquivos por Quadro Societario
        socio_pos_to_col(**kwargs)


In [6]:
def main():
    ### Organiza os arquivos
    organize_files()

if __name__ == '__main__':
    main()

Arquivo 20180718_AC.txt organizado por CNPJ.
Arquivo 20180718_AC.txt organizado por Quadro Societario.
Arquivo 20180718_AL.txt organizado por CNPJ.
Arquivo 20180718_AL.txt organizado por Quadro Societario.
Arquivo 20180718_AM.txt organizado por CNPJ.
Arquivo 20180718_AM.txt organizado por Quadro Societario.
Arquivo 20180718_AP.txt organizado por CNPJ.
Arquivo 20180718_AP.txt organizado por Quadro Societario.
Arquivo 20180718_BA.txt organizado por CNPJ.
Arquivo 20180718_BA.txt organizado por Quadro Societario.
Arquivo 20180718_CE.txt organizado por CNPJ.
Arquivo 20180718_CE.txt organizado por Quadro Societario.
Arquivo 20180718_DF.txt organizado por CNPJ.
Arquivo 20180718_DF.txt organizado por Quadro Societario.
Arquivo 20180718_ES.txt organizado por CNPJ.
Arquivo 20180718_ES.txt organizado por Quadro Societario.
Arquivo 20180718_GO.txt organizado por CNPJ.
Arquivo 20180718_GO.txt organizado por Quadro Societario.
Arquivo 20180718_MA.txt organizado por CNPJ.
Arquivo 20180718_MA.txt orga